In [1]:
# ultralytics installieren
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.8 MB/s eta 0:00:00


In [ ]:
# google drive mounten
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# rclone für onedrive  mounten
!curl https://rclone.org/install.sh | sudo bash
!rclone config

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0   7484      0 --:--:-- --:--:-- --:--:--  7478
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/rclone.1  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.72.1-linux-amd64/git-log.txt  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/zh_TW...
Processing manual pages under /usr/share/man/zh_TW...
Purging old database e

In [3]:
# Datei von onedrive kopieren
!rclone copy onedriveclone:/Dokumente/Blueberry9K.zip /content/


In [4]:
# unzip von google drive
!unzip Blueberry9K.zip -d dataset

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: dataset/train/labels/8072_jpg.rf.7a85a2e013bcfefe70c153a1cb2addd0.txt  
  inflating: dataset/train/labels/8079_jpg.rf.a6f8aacadaa5ad635f514fc729bcab04.txt  
  inflating: dataset/train/labels/807_jpg.rf.226e4ebc342ace96743076ccca5f89c1.txt  
  inflating: dataset/train/labels/8080_jpg.rf.3bbd123272166a6038f84048ea79a946.txt  
  inflating: dataset/train/labels/8084_jpg.rf.e7189459f7f8eca31170bb28fd99cb7e.txt  
  inflating: dataset/train/labels/8085_jpg.rf.874261b63ac91755d0e1fe8c8f8fa4f3.txt  
  inflating: dataset/train/labels/8086_jpg.rf.fda75aa27cf220a54ec28dbfd2e698fb.txt  
  inflating: dataset/train/labels/8088_jpg.rf.31bf89d700417aa40c9d45e64d52091b.txt  
  inflating: dataset/train/labels/808_jpg.rf.1457f655d67e2307825fe23b8826eb39.txt  
  inflating: dataset/train/labels/8091_jpg.rf.30ee847244ec08617499f121d23e2e9a.txt  
  inflating: dataset/train/labels/8092_jpg.rf.f857f4f02080d7afa7eb16073a7a9cdb.txt  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# splitten um Vaid und Testdaten zu erzeugen
import os
import random
import shutil
from glob import glob

# Pfade anpassen, falls nötig
image_dir = "dataset/train/images"
label_dir = "dataset/train/labels"

train_img = "dataset/images/train"
val_img = "dataset/images/val"
test_img = "dataset/images/test"

train_lbl = "dataset/labels/train"
val_lbl = "dataset/labels/val"
test_lbl = "dataset/labels/test"

# Zielordner erstellen
for d in [train_img, val_img, test_img, train_lbl, val_lbl, test_lbl]:
    os.makedirs(d, exist_ok=True)

# Alle Bilder sammeln
images = glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png"))
random.shuffle(images)

# Split-Verhältnis
train_split = 0.75
val_split = 0.2
test_split = 0.05  # optional

n = len(images)
train_end = int(n * train_split)
val_end = train_end + int(n * val_split)

train_files = images[:train_end]
val_files = images[train_end:val_end]
test_files = images[val_end:]

def copy_pair(img_path, target_img_dir, target_lbl_dir):
    shutil.copy(img_path, target_img_dir)
    lbl_path = img_path.replace("images", "labels").rsplit(".", 1)[0] + ".txt"
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, target_lbl_dir)

for f in train_files:
    copy_pair(f, train_img, train_lbl)

for f in val_files:
    copy_pair(f, val_img, val_lbl)

for f in test_files:
    copy_pair(f, test_img, test_lbl)

print("Fertig!")
print("Train:", len(train_files))
print("Val:", len(val_files))
print("Test:", len(test_files))

Fertig!
Train: 1848
Val: 492
Test: 124


In [ ]:
# data.yaml erzeugen
yaml_content = """
train: images/train
val: images/val
test: images/test

nc: 3
names: ["klasse1", "klasse2", "klasse3"]
"""

with open("dataset/data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml erstellt")

data.yaml erstellt


In [ ]:
# training durchführen
from ultralytics import YOLO

# Nano-Modell für mobile Geräte
model = YOLO("yolov8n.pt")
#    device=0,        # 0 = GPU in Colab

model.train(
    data="dataset/data.yaml",
    epochs=50,          # guter Ausgangspunkt für 9000 Bilder
    imgsz=640,           # Standardauflösung
    batch=12,             # stabil auf T4/P100, ggf. größer bei mehr RAM
    workers=2,
    device=0,            # GPU nutzen, sonst dauert es extrem lange
    patience=20,         # Early stopping
    optimizer="AdamW",   # schnelleres Konvergieren, robust bei größeren Datensätzen
    lr0=0.001,           # typische Start-LR für AdamW
    cos_lr=True          # sanftes Absenken der Lernrate über die Epochen
)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrain

In [ ]:
# modell runterladen
from google.colab import files
files.download('runs/detect/train3/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ergebnisse anzeigen
from IPython.display import Image
Image(filename='runs/detect/train3/results.png')

In [ ]:
# runs ordner aufräumen
!rm -rf /content/runs